# LangChain 核心模块 - Chat Model 和 Chat Prompt Template

希望通过此示例，让大家深入理解 LangChain 的聊天模型。简而言之：
- `Chat Model` 不止是一个用于聊天对话的模型抽象，更重要的是提供了`多角色`提示能力（System,AI,Human,Function)。
- `Chat Prompt Template` 则为开发者提供了便捷维护`不同角色`的`提示模板`与`消息记录`的接口。

![](../../jupyter/images/model_io.jpeg)


## 温故：LangChain Chat Model 使用方法和流程

在最终调用 `Chat Model` 时，一定是直接传入`LangChain Schema Messages（消息记录）` 

```python
from langchain.chat_models import ChatOpenAI
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo")

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

messages = [SystemMessage(content="You are a helpful assistant."),
 HumanMessage(content="Who won the world series in 2020?"),
 AIMessage(content="The Los Angeles Dodgers won the World Series in 2020."), 
 HumanMessage(content="Where was it played?")]

print(messages)

chat_model(messages)
```

打印 messages 输出结果：
```
[
    SystemMessage(content="You are a helpful assistant.", additional_kwargs={}),
    HumanMessage(
        content="Who won the world series in 2020?", additional_kwargs={}, example=False
    ),
    AIMessage(
        content="The Los Angeles Dodgers won the World Series in 2020.",
        additional_kwargs={},
        example=False,
    ),
    HumanMessage(content="Where was it played?", additional_kwargs={}, example=False),
]
```

调用 chat_model(messages) 返回结果：

```
AIMessage(
    content="The 2020 World Series was played at Globe Life Field in Arlington, Texas.",
    additional_kwargs={},
    example=False,
)

```

## 使用 System 和 Human 角色的提示模板构造 ChatPromptTemplate

使用 `ChatPromptTemplate.from_messages` 方法，类似使用和维护`messages`的方式，构造 `chat_prompt_template`

In [49]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
# 导入 Chat Model 即将使用的 Prompt Templates
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# 翻译任务指令始终由 System 角色承担
template = (
    """You are a translation expert, proficient in various languages. \n
    You are also an ancient chinese poet who can translate English to traditional Chinese poetically.
    Translates English to traditional Chinese."""
)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

In [50]:
print(system_message_prompt)

prompt=PromptTemplate(input_variables=[], template='You are a translation expert, proficient in various languages. \n\n    You are also an ancient chinese poet who can translate English to traditional Chinese poetically.\n    Translates English to traditional Chinese.')


In [51]:
# 待翻译文本由 Human 角色输入
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [54]:
print(human_message_prompt)

prompt=PromptTemplate(input_variables=['text'], template='{text}')


In [52]:
# 使用 System 和 Human 角色的提示模板构造 ChatPromptTemplate
chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

In [53]:
print(chat_prompt_template)

input_variables=['text'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a translation expert, proficient in various languages. \n\n    You are also an ancient chinese poet who can translate English to traditional Chinese poetically.\n    Translates English to traditional Chinese.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))]


### 规范化 Python 复杂对象

- 使用在线工具 [Python Formatter](https://codebeautify.org/python-formatter-beautifier) 
- 规范化 `chat_prompt_template`后再查看
- 注意：不要同事输入多个复杂对象

```json
messages = [
    SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=[],
            output_parser=None,
            partial_variables={},
            template="You are a translation expert, proficient in various languages. \n\n    Translates English to Chinese.",
            template_format="f-string",
            validate_template=True,
        ),
        additional_kwargs={},
    ),
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["text"],
            output_parser=None,
            partial_variables={},
            template="{text}",
            template_format="f-string",
            validate_template=True,
        ),
        additional_kwargs={},
    ),
]

```

In [55]:
# 生成用于翻译的 Chat Prompt
chat_prompt_template.format_prompt(text="I guess it comes down to a simple choice, really. Get busy living or get busy dying.")

ChatPromptValue(messages=[SystemMessage(content='You are a translation expert, proficient in various languages. \n\n    You are also an ancient chinese poet who can translate English to traditional Chinese poetically.\n    Translates English to traditional Chinese.'), HumanMessage(content='I guess it comes down to a simple choice, really. Get busy living or get busy dying.')])

## 使用 chat_prompt_template.to_messages 方法生成 Messages

In [56]:
# 生成聊天模型真正可用的消息记录 Messages
chat_prompt = chat_prompt_template.format_prompt(text="You are a piece of shit, damn!").to_messages()

In [57]:
chat_prompt

[SystemMessage(content='You are a translation expert, proficient in various languages. \n\n    You are also an ancient chinese poet who can translate English to traditional Chinese poetically.\n    Translates English to traditional Chinese.'),
 HumanMessage(content='You are a piece of shit, damn!')]

## 使用 Chat Model（GPT-3.5-turbo）实际执行翻译任务


In [58]:
from langchain.chat_models import ChatOpenAI

# 为了翻译结果的稳定性，将 temperature 设置为 0
translation_model = ChatOpenAI(api_key="sk-AlhxlLBU0BPYJD6I8870F34f377b47Dd8c1f86Ac73AfBe17",
    base_url="https://api.xiaoai.plus/v1", model_name="gpt-3.5-turbo", temperature=0)

In [59]:
translation_result = translation_model(chat_prompt)

In [60]:
translation_result

AIMessage(content='雖然怒火中燒，卻不願詛咒，\n心中不快，何妨隨風消散。', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 58, 'total_tokens': 90}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8ce9a5bd-210d-4e9e-b03b-85123aa10a5e-0')

In [61]:
# 查看翻译结果
print(translation_result.content)

雖然怒火中燒，卻不願詛咒，
心中不快，何妨隨風消散。


## 使用 LLMChain 简化重复构造 ChatPrompt

In [25]:
from langchain.chains import LLMChain

# 无需再每次都使用 to_messages 方法构造 Chat Prompt
translation_chain = LLMChain(llm=translation_model, prompt=chat_prompt_template)

D:\ProgramFiles\anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [62]:
# 等价于 translation_result.content (字符串类型)
chain_result = translation_chain.run({'text': "曾经有一份真挚的爱情摆在我的面前，我没有珍惜，等到失去的时候才后悔莫及。尘世间最痛苦的事莫过于此。如果上天能够给我一个再来一次的机会，我会对那个女孩子说三个字：我爱你。如果非要在这份爱上加上一个期限，我希望是……一万年。"})

In [63]:
print(chain_result)

Once there was a sincere love before me, which I did not cherish. When I lost it, I regretted it too late. There is no greater pain in the world than this. If heaven could give me another chance, I would say three words to that girl: I love you. If I must put a time limit on this love, I hope it would be... ten thousand years.


In [64]:
translation_chain.run({'text': "Once there was a sincere love before me, which I did not cherish. When I lost it, I regretted it too late. There is no greater pain in the world than this. If heaven could give me another chance, I would say three words to that girl: I love you. If I must put a time limit on this love, I hope it would be... ten thousand years."})

'曾经在我面前有一份真诚的爱，我却没有珍惜。当我失去它时，我后悔得太晚。世上没有比这更大的痛苦。如果上天能给我一次机会，我会对那个女孩说三个字：我爱你。如果我必须给这份爱设定一个期限，我希望是……一万年。'

In [18]:
translation_chain.run({'text': "[Fruit, Color, Price (USD)] [Apple, Red, 1.20] [Banana, Yellow, 0.50] [Orange, Orange, 0.80] [Strawberry, Red, 2.50] [Blueberry, Blue, 3.00] [Kiwi, Green, 1.00] [Mango, Orange, 1.50] [Grape, Purple, 2.00]"})


'[水果, 颜色, 价格（美元）] [苹果, 红色, 1.20] [香蕉, 黄色, 0.50] [橙子, 橙色, 0.80] [草莓, 红色, 2.50] [蓝莓, 蓝色, 3.00] [猕猴桃, 绿色, 1.00] [芒果, 橙色, 1.50] [葡萄, 紫色, 2.00]'

## 扩展：支持多语言对翻译

In [65]:
# System 增加 source_language 和 target_language
template = (
    """You are a translation expert, proficient in various languages. \n
    You speak with humor and charm, and with an artistic touch. \n 
    Translates {source_language} to {target_language}."""
)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

In [66]:
# 待翻译文本由 Human 角色输入
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [67]:
# 使用 System 和 Human 角色的提示模板构造 ChatPromptTemplate
m_chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

In [68]:
m_translation_chain = LLMChain(llm=translation_model, prompt=m_chat_prompt_template) 

In [69]:
m_translation_chain.run({
    "source_language": "English",
    "target_language": "Traditional Chinese",
    "text": "Once there was a sincere love before me, which I did not cherish. When I lost it, I regretted it too late. There is no greater pain in the world than this. If heaven could give me another chance, I would say three words to that girl: I love you. If I must put a time limit on this love, I hope it would be... ten thousand years.",
})

'曾經有一份真摯的愛在我面前，我卻未能珍惜。等我失去它時，為時已晚，遺憾如影隨形。世上沒有比這更大的痛苦。如果上天能給我一次機會，我會對那女孩說三個字：我愛你。如果我必須為這份愛設限，我希望是……一萬年。'

In [70]:
m_translation_chain.run({
    "source_language": "Korean",
    "target_language": "German",
    "text": "저는 노래, 춤, 랩, 농구를 좋아하는 차이 슈쿤입니다.",
})

'Ich bin Chai Shukun, und ich liebe Gesang, Tanz, Rap und Basketball! 🎤🏀💃 Lass uns die Bühne rocken!'

## Homework
- 尝试不同的 System Prompt 和 Chat Model，对比翻译效果。
- 根据翻译任务的使用场景，是否可以在初次传入 source_language 和 target_language 后不再更新？